In [91]:
#scrape urban turf

!pip3 install requests
import requests
!pip3 install bs4
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import certifi


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\asillers\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\asillers\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:

page = requests.get(
    "https://dc.urbanturf.com/pipeline?search_string=&project_type=0&status=0&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes&limit=1000")
soup = BeautifulSoup(page.content, 'html.parser')
page_body = soup.body

labels = [item for item in page_body.findAll("p")]
list_lab = [i.text.strip() for i in labels][1:] 
list_lab = [word.replace('\n','') for word in list_lab]

torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using 'torep' value as keys
pattern = re.compile("|".join(torep.keys())) #joins with "|"

desc = []

for i in range(0, len(list_lab)):
    a = list_lab[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #substitute the pattern in each value of the list with ""
    desc.append(text)

seq = range(0, len(desc))

urban_turf= pd.DataFrame(
    {"Address": [desc[i] for i in seq[0::5]],
     "Neighborhood": [desc[i] for i in seq[1::5]],
     "Project.Type":[desc[i] for i in seq[2::5]],
     "Status":[desc[i] for i in seq[3::5]],
     "Size":[desc[i] for i in seq[4::5]],
     "Pipeline Link": None,
     'Neighborhood:':None,
     'Pipeline':None,
     'Project type:': None, 
     'No. of units:': None, 
     'Types of units:': None, 
     'Unit sizes:':None, 
     'Amenities:':None, 
     'Pricing:':None, 
     'Website:':None, 
     'Architect:':None, 
     'Last updated:':None, 
     'Address:':None, 
     'Status:':None, 
     'First move-ins:':None, 
     "Latitude":None, 
     "Longitude":None})    

#can I get the pipeline link from scraping above?

def search_pipe_link(row):
    ut_address_col = str(row['Address'])
    projecttype_col = str(row['Project.Type'])
    status_col = str(row['Status'])
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + ut_address_col.replace(" ", "_" ) + "&project_type=" + projecttype_col.replace(" ", "+").replace("&", "%26") + "&status=" + status_col.replace(",", "%2C").replace(" ", "+") + "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    time.sleep(1.25)
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a').get('href')
        return tag
    else:
        return None

urban_turf['Pipeline Link'] = urban_turf.apply(search_pipe_link, axis=1)

'''
#do i still need this?
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

#if there is a pipeline link

def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in list(len(txt))] #need to expand range?
    txt = [txt[i][1] for i in list(len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)


'''
'''
for i in range(0, len(urban_turf.Address)):
    if type(urban_turf["Pipeline Link"][i]) == str:
    
        page = requests.get(urban_turf["Pipeline Link"][i])
        soup = BeautifulSoup(page.content, 'html.parser')
        page_body = soup.body
        table = soup.find_all('div', class_ = 'block') 
        txt= str(table)
        #every <span class> 
        #keep everything in between <span class="label">Project type:</span> and </p>

        txt = txt.replace('\r', '').replace('\n', '')

        #remove any instances where there are two spaces +
        for k in range(0,100):
            txt = txt.replace("  ", "")
        #remove all new lines
        txt = txt.split("<p>")  

        txt = [x for x in txt if '<span class="label">' in x] #this is the problem
        colnames = []
        values = []

        for j in range(0, len(txt)):
            a = txt[j].split("</span>")

            a[0] = re.sub('<span class="label">', "", a[0])

            colnames.append(a[0])

            a[1] = re.sub("</p>.*", "", a[1])
            a[1] = re.sub("\t{4,5}", "", a[1])
            a[1] = re.sub("<br/>", ",", a[1])
            a[1] = re.sub("\,.*DC", ", Washington, DC,", a[1])
            a[1] = re.sub('<a href="', "", a[1])
            a[1] = re.sub('" style.*', "", a[1])
            a[1] = re.sub('First', "1st", a[1])

            values.append(a[1])

        df_dictionary = pd.DataFrame([ dict(zip(colnames, values))])  #want to add this to urban_turf

        #urban_turf = pd.concat([urban_turf, df_dictionary],  ignore_index=True) #this adds it below instead of merging it 
        
        for key, value in df_dictionary.items():
            if key in column_names:
                urban_turf[key][i] = df_dictionary[key][0]
                  





#scrape pipeline links
for i in range(0, len(urban_turf.Address)):
    if type(urban_turf["Pipeline Link"][i]) == str:
    
        page = requests.get(urban_turf["Pipeline Link"][i])
        soup = BeautifulSoup(page.content, 'html.parser')
        page_body = soup.body
        table = soup.find_all('div', class_ = 'block') 
        txt= str(table)
        #every <span class> 
        #keep everything in between <span class="label">Project type:</span> and </p>

        txt = txt.replace('\r', '').replace('\n', '')

        #remove any instances where there are two spaces +
        for k in range(0,100):
            txt = txt.replace("  ", "")
        #remove all new lines
        txt = txt.split("<p>")  

        txt = [x for x in txt if '<span class="label">' in x] #this is the problem
        colnames = []
        values = []

        for j in range(0, len(txt)):
            a = txt[j].split("</span>")

            a[0] = re.sub('<span class="label">', "", a[0])

            colnames.append(a[0])

            a[1] = re.sub("</p>.*", "", a[1])
            a[1] = re.sub("\t{4,5}", "", a[1])
            a[1] = re.sub("<br/>", ",", a[1])
            a[1] = re.sub("\,.*DC", ", Washington, DC,", a[1])
            a[1] = re.sub('<a href="', "", a[1])
            a[1] = re.sub('" style.*', "", a[1])
            a[1] = re.sub('First', "1st", a[1])

            values.append(a[1])

        df_dictionary = pd.DataFrame([ dict(zip(colnames, values))])  #want to add this to urban_turf

        #urban_turf = pd.concat([urban_turf, df_dictionary],  ignore_index=True) #this adds it below instead of merging it 
        
        for key, value in df_dictionary.items():
            if key in column_names:
                urban_turf[key][i] = df_dictionary[key][0]
                  
    else:
        continue

def get_lat_long(address):
    geolocator = Nominatim(user_agent="my_geocoder_app")
    time.sleep(1.1)
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return None
    
urban_turf['Latitude'] = ""
urban_turf['Longitude'] = ""

for i in range(0, len(urban_turf)):
    try:
        c = urban_turf['Address:'][i]
        d = re.sub("-[\d]*", "", c)
        a = get_lat_long(d)
        urban_turf['Latitude'][i] =  a[0]
        urban_turf['Longitude'][i] =  a[1]

    except Exception:
        pass
urban_turf['No. of units:'] = pd.to_numeric(urban_turf['No. of units:'], errors='coerce')
'''

'\nfor i in range(0, len(urban_turf.Address)):\n    if type(urban_turf["Pipeline Link"][i]) == str:\n    \n        page = requests.get(urban_turf["Pipeline Link"][i])\n        soup = BeautifulSoup(page.content, \'html.parser\')\n        page_body = soup.body\n        table = soup.find_all(\'div\', class_ = \'block\') \n        txt= str(table)\n        #every <span class> \n        #keep everything in between <span class="label">Project type:</span> and </p>\n\n        txt = txt.replace(\'\r\', \'\').replace(\'\n\', \'\')\n\n        #remove any instances where there are two spaces +\n        for k in range(0,100):\n            txt = txt.replace("  ", "")\n        #remove all new lines\n        txt = txt.split("<p>")  \n\n        txt = [x for x in txt if \'<span class="label">\' in x] #this is the problem\n        colnames = []\n        values = []\n\n        for j in range(0, len(txt)):\n            a = txt[j].split("</span>")\n\n            a[0] = re.sub(\'<span class="label">\', "", a

In [781]:
def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]

    key = [txt[i][0] for i in list(range(0,len(txt)))] #need to expand range?
    txt = [txt[i][1] for i in list(range(0,len(txt)))]
    #key = [x.replace('<span class="label">', "") for x in key]
    #df_dictionary = pd.DataFrame([ dict(zip(key, txt))])

    
    #for key, txt in df_dictionary.items():
    #        if key in column_names:
    #           urban_turf[key][row] = df_dictionary[key][0]

#urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)

In [788]:
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

#if there is a pipeline link

def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in list(len(txt))] #need to expand range?
    txt = [txt[i][1] for i in list(len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

urban_turf = urban_turf.apply(scrape_pipelinelink, axis=1)

KeyError: "None of [Index([                                       'Valen',\n                                   'National Landing',\n                                             'Rental',\n                              'Delivered, Leasing Up',\n                                         '338  units',\n       'https://dc.urbanturf.com/pipeline/1255/Valen',\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                                                 None,\n                            

In [ ]:
def scrape_pipelinelink(row):
    page = requests.get(urban_turf["Pipeline Link"][row])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]
    key = [txt[i][0] for i in range(0, len(txt))] 
    txt = [txt[i][1] for i in range(0, len(txt))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])
    for key, txt in df_dictionary.items():
            if key in column_names:
               urban_turf[key][row] = df_dictionary[key][0]

In [838]:
urban_turf.iloc[20]  = scrape_pipelinelink(20)

In [845]:
a = scrape_pipelinelink(50)

In [846]:
for key, txt in a.items():
    if key in column_names:
        urban_turf[key][20] = a[key][0]

In [822]:
key = [a[i][0] for i in range(0,len(a))]

In [847]:
urban_turf.iloc[20]

Address                                                         None
Neighborhood                                                    None
Project.Type                                                    None
Status                                                          None
Size                                                            None
Pipeline Link                                                   None
Neighborhood:                                                   None
Pipeline                                                        None
Project type:                                        Condo or Ren...
No. of units:                                364                </p>
Types of units:                              TBD                </p>
Unit sizes:                                                     None
Amenities:                                                      None
Pricing:                                                        None
Website:           <a href="https:

In [710]:
a

['Project type:',
 'No. of units:',
 'Types of units:',
 'Website:',
 'Last updated:',
 'Address:',
 'Status:']

In [707]:
for i in range(0, len(a)):
    a[i] = a[i].replace('<span class="label">', "")

In [708]:
a[0]

'Project type:'

In [694]:
urban_turf['No. of units:'] = a[1]

In [721]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,Valen,National Landing,Rental,"Delivered, Leasing Up",338 units,https://dc.urbanturf.com/pipeline/1255/Valen,None,None,Rental apart...,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
1,Bryant Street Development,Edgewood,Rental,"Under Construction, Leasing Up",1650 units,https://dc.urbanturf.com/pipeline/457/Bryant_S...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
2,Fox 5 Headquarters Redevelopment,Friendship Heights,Rental,"Under Construction, Not Yet Leasing",210 units,https://dc.urbanturf.com/pipeline/834/Fox_5_He...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
3,Barry Farm Redevelopment,Barry Farm,Condos & Rental apartments,"Under Construction, Leasing Up","1,000",https://dc.urbanturf.com/pipeline/579/Barry_Fa...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
4,Kiley Apartments,Southwest,Rental,"Delivered, Leasing Up",330 units,https://dc.urbanturf.com/pipeline/446/Kiley_Ap...,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,https://dc.urbanturf.com/pipeline/383/The_Hendrix,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
673,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,None,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
674,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,https://dc.urbanturf.com/pipeline/468/The_Pearl,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None
675,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,None,None,None,None,1650 </p>,...,None,None,None,None,None,None,None,None,None,None


urban_turf

In [670]:
[a[i][1] for i in list(range(0,7))]

['                                  Rental apartments                              </p>',
 '                  338                </p>',
 '                  Junior one-bedrooms, one-, two- and three-bedroom apartments                </p>',
 '<a href="https://www.thezoevalen.com/valen" style="text-decoration: underline;" target="_blank" title="">www.thezoevalen.com</a>\xa0\t\t\t\t\t\t\t\t</p>',
 '                October 22nd, 2025              </p></div>, <div class="block"><h3>Where?</h3>',
 '                  2000 S Bell St                                                        <br/> Arlington,\xa0VA                                                        \xa022202                                  </p><div class="map-wrapper"><iframe frameborder="0" height="350" marginheight="0" marginwidth="0" scrolling="no" src="https://maps.google.com/maps?hl=en&amp;q=2000+S+Bell+St+%2C+Arlington+22202&amp;ie=UTF8&amp;z=15&amp;iwloc=&amp;output=embed" width="410"></iframe></div><div style="clear:righ

In [668]:
b = [0,1,2,3,4,5,6,7]

In [645]:
b = [1, 3, 5, 7]

In [637]:
for i in b:
    print(a[i])

<span class="label">No. of units:</span>                  1650                </p>
<span class="label">Amenities:</span>                  Approx. 2,000 parking spaces, a two-level movie theater, pedestrian bridge to Rhode Island Avenue Metro Station                </p>
<span class="label">Architect:</span>					SK+I Architecture				  </p>
<span class="label">Address:</span>                  680 Rhode Island Avenue NE                                                        <br/> Washington, DC                                                         20002                                  </p><div class="map-wrapper"><iframe frameborder="0" height="350" marginheight="0" marginwidth="0" scrolling="no" src="https://maps.google.com/maps?hl=en&amp;q=680+Rhode+Island+Avenue+NE+%2C+Washington+20002&amp;ie=UTF8&amp;z=15&amp;iwloc=&amp;output=embed" width="410"></iframe></div><div style="clear:right"></div></div>, <div class="block last"><h3>When?</h3>


IndexError: list index out of range

In [608]:
a[1]

'<span class="label">Project type:</span>                                  Rental apartments                              </p>'

In [609]:
a[2]

'<span class="label">No. of units:</span>                  1650                </p>'

In [603]:
values

['<', 's', 's', 's', 's', 's', 's', 's', 's', 's', 'W', '0', 'h', '0', 'M']

In [532]:
colnames = []
values = []

df_dictionary = pd.DataFrame([ dict(zip(colnames, values))])  #want to add this to urban_turf


In [544]:
txt

[['<span class="label">Project type:', 'Rental apartments</p>'],
 ['<span class="label">No. of units:', '262</p>'],
 ['<span class="label">Types of units:',
  'Studios, one-, two- and three-bedroom apartments</p>'],
 ['<span class="label">Unit sizes:', '630 - 1,477 square feet</p>'],
 ['<span class="label">Amenities:',
  'Resort-style pool with active communal courtyard, fitness area with golf simulator, business center, theater room</p>'],
 ['<span class="label">Pricing:', '$1,710 - $2,920</p>'],
 ['<span class="label">Website:',
  '<a href="http://www.esplanadenationalharbor.com/?utm_source=bozzuto.com&amp;utm_medium=url&amp;utm_campaign=bozzuto.com" style="text-decoration: underline;" target="_blank" title="">www.esplanadenationalharbor.com</a>\xa0\t\t\t\t\t\t\t\t</p>'],
 ['<span class="label">Architect:',
  '\t\t\t\t\tHeffner Architects\t\t\t\t</p>'],
 ['<span class="label">Last updated:',
  'September 18th, 2022</p></div>, <div class="block"><h3>Where?</h3>'],
 ['<span class="labe

In [545]:
values = [txt[1] for txt in txt]

In [546]:
values

['Rental apartments</p>',
 '262</p>',
 'Studios, one-, two- and three-bedroom apartments</p>',
 '630 - 1,477 square feet</p>',
 'Resort-style pool with active communal courtyard, fitness area with golf simulator, business center, theater room</p>',
 '$1,710 - $2,920</p>',
 '<a href="http://www.esplanadenationalharbor.com/?utm_source=bozzuto.com&amp;utm_medium=url&amp;utm_campaign=bozzuto.com" style="text-decoration: underline;" target="_blank" title="">www.esplanadenationalharbor.com</a>\xa0\t\t\t\t\t\t\t\t</p>',
 '\t\t\t\t\tHeffner Architects\t\t\t\t</p>',
 'September 18th, 2022</p></div>, <div class="block"><h3>Where?</h3>',
 '250 American Way<br/> Oxon Hill,\xa0MD\xa020745</p><div class="map-wrapper"><iframe frameborder="0" height="350" marginheight="0" marginwidth="0" scrolling="no" src="https://maps.google.com/maps?hl=en&amp;q=250+American+Way+%2C+Oxon+Hill+20745&amp;ie=UTF8&amp;z=15&amp;iwloc=&amp;output=embed" width="410"></iframe></div><div style="clear:right"></div></div>, <di

In [ ]:
for key, value in df_dictionary.items():
    if key in column_names:
        urban_turf[key][i] = df_dictionary[key][0]


In [535]:
#a = txt[j].split("</span>")

txt = [txt.split("</span>") for txt in txt]


In [ ]:
txt

'630 - 1,477 square feet</p>'

In [438]:
urban_turf["Pipeline Link"][0] = search_pipe_link(urban_turf.head(1))


KeyError: "None of [Index(['https://dc.urbanturf.com/pipeline?search_string=Valen&project_type=Rental&status=Delivered%2C+Leasing+Up&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes',\n       'https://dc.urbanturf.com/pipeline?search_string=Bryant_Street_Development&project_type=Rental&status=Under+Construction%2C+Leasing+Up&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes',\n       'https://dc.urbanturf.com/pipeline?search_string=Fox_5_Headquarters_Redevelopment&project_type=Rental&status=Under+Construction%2C+Not+Yet+Leasing&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes',\n       'https://dc.urbanturf.com/pipeline?search_string=Barry_Farm_Redevelopment&project_type=Condos+%26+Rental+apartments&status=Under+Construction%2C+Leasing+Up&more_or_l

In [ ]:
def search_pipe_line(search, projecttype, status):
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + search + "&project_type=" + projecttype + "&status=" + status + "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a').get('href')
        return tag
    else:
        return None

In [ ]:
','.join(str(i) for i in xrange(700) if i % 4 in (1,2))

TypeError: 'list' object is not callable

In [338]:
status = urban_turf['Status'].replace(",", "%2C").str.replace(" ", "+")


In [317]:
ut_address = ut_address.replace(" ", "_")

In [321]:
ut_address[1].replace(" ", "_")

'Bryant_Street_Development'

In [ ]:
for x in list_a:
    if x % 5 == 0:
        print(x)


list_b = [x for x in list_a if x % 5 == 0]        

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95


In [306]:
list_b = [x for x in list_a if x % 5 == 0]

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]

In [ ]:
#seq = []


#for i in range(0, len(desc)):
#    if i % 5 == 1:
#        seq.append(i)

#seq2 = [x for i, x in enumerate(desc) if i % 5 == 1]
#labels = [item for item in page_body.findAll("p")]


In [270]:
urban_turf

,Address,Neighborhood,Project.Type,Status,Size,Pipeline Link,Neighborhood:,Pipeline,Project type:,No. of units:,...,Amenities:,Pricing:,Website:,Architect:,Last updated:,Address:,Status:,First move-ins:,Latitude,Longitude
0,Valen,National Landing,Rental,"Delivered, Leasing Up",338 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Bryant Street Development,Edgewood,Rental,"Under Construction, Leasing Up",1650 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Fox 5 Headquarters Redevelopment,Friendship Heights,Rental,"Under Construction, Not Yet Leasing",210 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,Barry Farm Redevelopment,Barry Farm,Condos & Rental apartments,"Under Construction, Leasing Up","1,000",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Kiley Apartments,Southwest,Rental,"Delivered, Leasing Up",330 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,The Hendrix,Trinidad,Rental,"Delivered, Leasing Up",45 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
673,Latitude Apartments,Virginia Square,Rental,"Delivered, Leasing Up",265 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
674,The Pearl,Silver Spring,Rental,"Delivered, Leasing Up",507 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
675,Ten at Clarendon,Clarendon,Rental,"Delivered, Leasing Up",144 units,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [235]:
desc

['Valen',
 'National Landing',
 'Rental',
 'Delivered, Leasing Up',
 '338  units',
 'Bryant Street Development',
 'Edgewood',
 'Rental',
 'Under Construction, Leasing Up',
 '1650  units',
 'Fox 5 Headquarters Redevelopment',
 'Friendship Heights',
 'Rental',
 'Under Construction, Not Yet Leasing',
 '210  units',
 'Barry Farm Redevelopment',
 'Barry Farm',
 'Condos & Rental apartments',
 'Under Construction, Leasing Up',
 '1,000',
 'Kiley Apartments',
 'Southwest',
 'Rental',
 'Delivered, Leasing Up',
 '330  units',
 'DCHA Headquarters Redevelopment',
 'NoMa',
 'Rental',
 'Under Construction, Leasing Up',
 '1,100',
 'Terrace Manor',
 'Randle Heights',
 'Rental',
 'Delivered, Leasing Up',
 '130  units',
 'Solaire 7607',
 'Downtown Bethesda',
 'Rental',
 'Delivered, Leasing Up',
 '198  units',
 'EucKal',
 'Adams Morgan',
 'Rental',
 'Delivered, Leasing Up',
 '25  units',
 'Hill East Development District',
 'Hill East',
 'Rental',
 'Under Construction, Not Yet Leasing',
 '1,068',
 'Rosslyn

In [249]:
location = [desc[i] for i in xs[0::5]]

In [507]:
urban_turf['Pipeline Link'][0]

'https://dc.urbanturf.com/pipeline/1255/Valen'

In [ ]:
location = [desc[i] for i in seq]

677

In [395]:
import pandas as pd

# Create a sample DataFrame
data = {'name': ['Alice', 'Bob', 'Charlie'],
        'age': [25, 30, 35],
        'city': ['New York', 'London', 'Paris']}
df = pd.DataFrame(data)

# Define a function that uses column values from each row
def describe_person(row):
    return f"{row['name']} is {row['age']} years old and lives in {row['city']}."

# Apply the function to each row and store the result in a new column
df['description'] = df.apply(describe_person, axis=1)

print(df)

      name  age      city                                   description
0    Alice   25  New York  Alice is 25 years old and lives in New York.
1      Bob   30    London      Bob is 30 years old and lives in London.
2  Charlie   35     Paris   Charlie is 35 years old and lives in Paris.


In [ ]:
#lambda m:

torep[re.escape(m.group(0))]


In [167]:
list[2]

'Project type:            Condo'

In [ ]:
torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using value as keys
pattern = re.compile("|".join(torep.keys())) #joins

for i in range(0, len(list)):
    a = list[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #re.escape - remove extra values, #substitute the pattern in each value of the list (a)
    desc.append(text)

In [184]:
text

'262  units'

In [187]:
list

['Valen',
 'Location:            National Landing',
 'Project type:            Rental',
 'Status:            Delivered, Leasing Up',
 'Size:            338  units',
 'Bryant Street Development',
 'Location:            Edgewood',
 'Project type:            Rental',
 'Status:            Under Construction, Leasing Up',
 'Size:            1650  units',
 'Fox 5 Headquarters Redevelopment',
 'Location:            Friendship Heights',
 'Project type:            Rental',
 'Status:            Under Construction, Not Yet Leasing',
 'Size:            210  units',
 'Barry Farm Redevelopment',
 'Location:            Barry Farm',
 'Project type:            Condos & Rental apartments',
 'Status:            Under Construction, Leasing Up',
 'Size:            1,000',
 'Kiley Apartments',
 'Location:            Southwest',
 'Project type:            Rental',
 'Status:            Delivered, Leasing Up',
 'Size:            330  units',
 'DCHA Headquarters Redevelopment',
 'Location:            NoMa',
 'P

In [183]:
def ads(x):
    return(lambda y:x+y)(4)
print(ads(8))


12


In [41]:
with zipfile.ZipFile("C:/Users/asillers/OneDrive - The American Council of Trustees Alumni/Metro_Lines_Regional.zip", 'r') as zip_ref:
    zip_ref.extractall("C:/Users/asillers/OneDrive - The American Council of Trustees Alumni")

In [ ]:
ridership_metro = pd.read_csv("C:/Users/asillers/OneDrive - The American Council of Trustees Alumni/Daily Ridership - Station-Level Line Chart_Full Da_data.csv")
metro_lat_lon = pd.read_csv("C:/Users/asillers/OneDrive - The American Council of Trustees Alumni/metro_lat_lon.csv")
ridership_metro = pd.merge(ridership_metro, metro_lat_lon, on='Station Name', how='inner') # inner join (default)
ridership_metro = ridership_metro.groupby(['Station Name']).mean('Entries Or Boardings')


In [35]:
ridership_metro = ridership_metro.groupby(['Station Name']).mean('Entries Or Boardings')


In [188]:
labels

[<p><strong>What's Hot: </strong><a href="https://dc.urbanturf.com/articles/blog/what_dcs_rental_act_changes_for_tenants_landlords_and_housing/23990">What DC’s RENTAL Act Changes for Tenants, Landlords, and Housing</a></p>,
 <p>
 <h2>
 <a href="https://dc.urbanturf.com/pipeline/1255/Valen" title="Valen">
             Valen
             </a>
 </h2>
 </p>,
 <p>
 <span class="label">Location:</span>
             National Landing
         </p>,
 <p>
 <span class="label">Project type:</span>
             Rental
         </p>,
 <p>
 <span class="label">Status:</span>
             Delivered, Leasing Up
         </p>,
 <p>
 <span class="label">Size:</span>
             338  units          </p>,
 <p>
 <h2>
 <a href="https://dc.urbanturf.com/pipeline/457/Bryant_Street_Development" title="Bryant Street Development">
             Bryant Street Development
             </a>
 </h2>
 </p>,
 <p>
 <span class="label">Location:</span>
             Edgewood
         </p>,
 <p>
 <span class="label">Projec

In [87]:
labels = page_body.findAll("p")
